# Circuit Parameter Manipulation and Ultrafast Derivatives

This tutorial covers the common use case of constructing a circuit with continuously variable parameters (today only in `Rx`, `Ry`, or `Rz` gates), evaluating the expectation value of a Hermitian operator supplied in sparse Pauli form using ideal statevector simulation, and then taking the gradient of the expectation value with respect to the values of the circuit rotation angle parameters. Such exercises are ubiquitous in variational quantum algorithms such as QAOA and VQE, e.g., in gradient-based algorithms to optimize the variational quantum circuit parameters. We have provided some utility functions to make these exercises easy to implement and rather fast to run (particularly when ideal statevector simulation in the infinite statistical sampling limit is invoked).

In [1]:
import quasar
import time # We'll roughly time some quasar operations below

## Case-Study Circuit

First, let's build a CIS state preparation circuit. We start by building a gadget to move amplitude from one singly-excited ket to another:

In [2]:
gadget = quasar.Circuit().Ry(1).CZ(0,1).Ry(1).CX(1,0)
print(gadget)

T  : |0 |1|2 |3|

q0 : ----@----X-
         |    | 
q1 : -Ry-Z-Ry-@-
                
T  : |0 |1|2 |3|



Then use the ``add_gates`` utility function to quickly build up the full CIS state preparation circuit:

In [21]:
N = 19 # Increase this to get some more dramatic timing examples below
circuit = quasar.Circuit().X(0)
for I in range(N):
    circuit.add_gates(circuit=gadget, qubits=(I, I+1))
print(circuit)

T   : |0 |1|2 |3 |4|5 |6 |7|8 |9 |10|11|12|13|14|15|16|17|18|19|20|21|22|23|24|

q0  : -X--@----X---------------------------------------------------------------
          |    |                                                               
q1  : -Ry-Z-Ry-@--@----X-------------------------------------------------------
                  |    |                                                       
q2  : ---------Ry-Z-Ry-@--@----X-----------------------------------------------
                          |    |                                               
q3  : -----------------Ry-Z-Ry-@--@-----X--------------------------------------
                                  |     |                                      
q4  : -------------------------Ry-Z--Ry-@--@-----X-----------------------------
                                           |     |                             
q5  : ----------------------------------Ry-Z--Ry-@--@-----X--------------------
                                       

## Parameter Manipulation

Cool circuit, bro. So where are all the parameters, how do we figure out what they are, and how do we set their values? To get started with this, let's invoke the `parameter_str` utility property of `Circuit`:

In [22]:
print(circuit.parameter_str)

Index Time       Qubits     Name       Gate      :     Value
0     (0,)       (1,)       theta      Ry        :  0.000000
1     (2,)       (1,)       theta      Ry        :  0.000000
2     (3,)       (2,)       theta      Ry        :  0.000000
3     (5,)       (2,)       theta      Ry        :  0.000000
4     (6,)       (3,)       theta      Ry        :  0.000000
5     (8,)       (3,)       theta      Ry        :  0.000000
6     (9,)       (4,)       theta      Ry        :  0.000000
7     (11,)      (4,)       theta      Ry        :  0.000000
8     (12,)      (5,)       theta      Ry        :  0.000000
9     (14,)      (5,)       theta      Ry        :  0.000000
10    (15,)      (6,)       theta      Ry        :  0.000000
11    (17,)      (6,)       theta      Ry        :  0.000000
12    (18,)      (7,)       theta      Ry        :  0.000000
13    (20,)      (7,)       theta      Ry        :  0.000000
14    (21,)      (8,)       theta      Ry        :  0.000000
15    (23,)      (8,)   

This provides a textual summary of all mutable parameters in `Circuit`, including a breakdown of the `times` and `qubits` key of the involved `Gate`, the name of that `Gate`, and the parameter name within that `Gate`, as well as the current value of the parameter. The parameters of a `Circuit` object have a strong lexical indexing order (referred to as the `parameter_index` order) determined first by the `times` and `qubits` key of the involved `Gate` (i.e., the underlying lexical `Gate` index order), followed by the definitional ordering of each parameter within the `Gate` (for multi-parameter `Gate` objects). For instance, the `10`-th `parameter_index` in our `Circuit` corresponds to the `theta` parameter of the `Ry` gates at `time` index `15` and `qubit` index `6` - its value is currently `0.0`.

Before we start setting parameters, let's look at some more attributes that help us to locate and characterize these parameters. The `nparameter` property tells how many total parameters are in the `Circuit` object:

In [23]:
print(circuit.nparameter)

38


The `parameters` property is an `OrderedDict` of parameter (`times`, `qubits`, `parameter_key`) keys to parameter values. We'll print this out somewhat reasonably here: 

In [24]:
# print(circuit.parameters)
for key, value in circuit.parameters.items():
    times, qubits, parameter_key = key
    print('%10r %10r %10r: %8.6f' % (times, qubits, parameter_key, value))

      (0,)       (1,)    'theta': 0.000000
      (2,)       (1,)    'theta': 0.000000
      (3,)       (2,)    'theta': 0.000000
      (5,)       (2,)    'theta': 0.000000
      (6,)       (3,)    'theta': 0.000000
      (8,)       (3,)    'theta': 0.000000
      (9,)       (4,)    'theta': 0.000000
     (11,)       (4,)    'theta': 0.000000
     (12,)       (5,)    'theta': 0.000000
     (14,)       (5,)    'theta': 0.000000
     (15,)       (6,)    'theta': 0.000000
     (17,)       (6,)    'theta': 0.000000
     (18,)       (7,)    'theta': 0.000000
     (20,)       (7,)    'theta': 0.000000
     (21,)       (8,)    'theta': 0.000000
     (23,)       (8,)    'theta': 0.000000
     (24,)       (9,)    'theta': 0.000000
     (26,)       (9,)    'theta': 0.000000
     (27,)      (10,)    'theta': 0.000000
     (29,)      (10,)    'theta': 0.000000
     (30,)      (11,)    'theta': 0.000000
     (32,)      (11,)    'theta': 0.000000
     (33,)      (12,)    'theta': 0.000000
     (35,) 

If you don't want the whole `parameters` dictionary, you can also have either the `parameter_keys` or `parameter_values` in isolation:

In [25]:
print(circuit.parameter_keys)
print(circuit.parameter_values)

[((0,), (1,), 'theta'), ((2,), (1,), 'theta'), ((3,), (2,), 'theta'), ((5,), (2,), 'theta'), ((6,), (3,), 'theta'), ((8,), (3,), 'theta'), ((9,), (4,), 'theta'), ((11,), (4,), 'theta'), ((12,), (5,), 'theta'), ((14,), (5,), 'theta'), ((15,), (6,), 'theta'), ((17,), (6,), 'theta'), ((18,), (7,), 'theta'), ((20,), (7,), 'theta'), ((21,), (8,), 'theta'), ((23,), (8,), 'theta'), ((24,), (9,), 'theta'), ((26,), (9,), 'theta'), ((27,), (10,), 'theta'), ((29,), (10,), 'theta'), ((30,), (11,), 'theta'), ((32,), (11,), 'theta'), ((33,), (12,), 'theta'), ((35,), (12,), 'theta'), ((36,), (13,), 'theta'), ((38,), (13,), 'theta'), ((39,), (14,), 'theta'), ((41,), (14,), 'theta'), ((42,), (15,), 'theta'), ((44,), (15,), 'theta'), ((45,), (16,), 'theta'), ((47,), (16,), 'theta'), ((48,), (17,), 'theta'), ((50,), (17,), 'theta'), ((51,), (18,), 'theta'), ((53,), (18,), 'theta'), ((54,), (19,), 'theta'), ((56,), (19,), 'theta')]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0

In another common use case, you have the `times` and `qubits` key of a `Gate` object, and want to know what parameter indices it covers in the `Circuit` object. For this, the `parameter_indices` property is provided:

In [26]:
# print(circuit.parameter_indices)
for key, parameter_indices in circuit.parameter_indices.items():
    times, qubits = key
    print('%10r %10r: %r' % (times, qubits, parameter_indices))

      (0,)       (0,): ()
      (0,)       (1,): (0,)
      (1,)     (0, 1): ()
      (2,)       (1,): (1,)
      (3,)     (1, 0): ()
      (3,)       (2,): (2,)
      (4,)     (1, 2): ()
      (5,)       (2,): (3,)
      (6,)     (2, 1): ()
      (6,)       (3,): (4,)
      (7,)     (2, 3): ()
      (8,)       (3,): (5,)
      (9,)     (3, 2): ()
      (9,)       (4,): (6,)
     (10,)     (3, 4): ()
     (11,)       (4,): (7,)
     (12,)     (4, 3): ()
     (12,)       (5,): (8,)
     (13,)     (4, 5): ()
     (14,)       (5,): (9,)
     (15,)     (5, 4): ()
     (15,)       (6,): (10,)
     (16,)     (5, 6): ()
     (17,)       (6,): (11,)
     (18,)     (6, 5): ()
     (18,)       (7,): (12,)
     (19,)     (6, 7): ()
     (20,)       (7,): (13,)
     (21,)     (7, 6): ()
     (21,)       (8,): (14,)
     (22,)     (7, 8): ()
     (23,)       (8,): (15,)
     (24,)     (8, 7): ()
     (24,)       (9,): (16,)
     (25,)     (8, 9): ()
     (26,)       (9,): (17,)
     (27,)     (9, 8

Note how the `H`, `CX`, and `CZ` gates do not have any parameters, while the `Ry` gates have one each. The `parameter_indices` method can often be used in concert with automatic time placement in the `add_gate` method and the `return_key` optional argument to `add_gate` to determine where each `Gate` object is automatically added in time and from thence which lexical parameter indices correspond to that `Gate`. `return_key` falls through any gate addition sugar methods. For instance:

In [27]:
circuit2 = quasar.Circuit().Rx(0).Ry(1)
# Where is the following Rz gate added? 
# (Note that time placement is determined automatically)
Rz_times, Rz_qubits = circuit2.Rz(0, return_key=True)
print(circuit2)
print(Rz_times, Rz_qubits)
# Maybe add more gates here
# ...
# At this point, what parameter indices are represented by the Rz gate?
print(circuit2.parameter_indices[(Rz_times, Rz_qubits)])

T  : |0 |1 |

q0 : -Rx-Rz-
            
q1 : -Ry----
            
T  : |0 |1 |

(1,) (0,)
(2,)


Now to set the values of the parameters. You can set these one at a time with full knowledge of the parameter key via the `set_parameter` method:

In [28]:
circuit.set_parameter(((23,), (8,), 'theta'), 1.0)
print(circuit.parameter_str)

Index Time       Qubits     Name       Gate      :     Value
0     (0,)       (1,)       theta      Ry        :  0.000000
1     (2,)       (1,)       theta      Ry        :  0.000000
2     (3,)       (2,)       theta      Ry        :  0.000000
3     (5,)       (2,)       theta      Ry        :  0.000000
4     (6,)       (3,)       theta      Ry        :  0.000000
5     (8,)       (3,)       theta      Ry        :  0.000000
6     (9,)       (4,)       theta      Ry        :  0.000000
7     (11,)      (4,)       theta      Ry        :  0.000000
8     (12,)      (5,)       theta      Ry        :  0.000000
9     (14,)      (5,)       theta      Ry        :  0.000000
10    (15,)      (6,)       theta      Ry        :  0.000000
11    (17,)      (6,)       theta      Ry        :  0.000000
12    (18,)      (7,)       theta      Ry        :  0.000000
13    (20,)      (7,)       theta      Ry        :  0.000000
14    (21,)      (8,)       theta      Ry        :  0.000000
15    (23,)      (8,)   

Or several at a time with the `set_parameters` method:

In [29]:
circuit.set_parameters({
    ((20,), (7,), 'theta') : 0.2,
    ((21,), (8,), 'theta') : 0.3,
    })
print(circuit.parameter_str)

Index Time       Qubits     Name       Gate      :     Value
0     (0,)       (1,)       theta      Ry        :  0.000000
1     (2,)       (1,)       theta      Ry        :  0.000000
2     (3,)       (2,)       theta      Ry        :  0.000000
3     (5,)       (2,)       theta      Ry        :  0.000000
4     (6,)       (3,)       theta      Ry        :  0.000000
5     (8,)       (3,)       theta      Ry        :  0.000000
6     (9,)       (4,)       theta      Ry        :  0.000000
7     (11,)      (4,)       theta      Ry        :  0.000000
8     (12,)      (5,)       theta      Ry        :  0.000000
9     (14,)      (5,)       theta      Ry        :  0.000000
10    (15,)      (6,)       theta      Ry        :  0.000000
11    (17,)      (6,)       theta      Ry        :  0.000000
12    (18,)      (7,)       theta      Ry        :  0.000000
13    (20,)      (7,)       theta      Ry        :  0.200000
14    (21,)      (8,)       theta      Ry        :  0.300000
15    (23,)      (8,)   

However, full knowledge of the parameter key is somewhat verbose, and is entirely equivalent to knowledge of the integer parameter index. To set parameters instead in terms of parameter index, we provide the `set_parameter_values` method, which takes a `parameter_values` list and an optional `parameter_indices` list (needed if only a subset and/or a number of non-sequential parameters are set):

In [30]:
circuit.set_parameter_values([0.4, 0.5], parameter_indices=[4,2])
print(circuit.parameter_str)

Index Time       Qubits     Name       Gate      :     Value
0     (0,)       (1,)       theta      Ry        :  0.000000
1     (2,)       (1,)       theta      Ry        :  0.000000
2     (3,)       (2,)       theta      Ry        :  0.500000
3     (5,)       (2,)       theta      Ry        :  0.000000
4     (6,)       (3,)       theta      Ry        :  0.400000
5     (8,)       (3,)       theta      Ry        :  0.000000
6     (9,)       (4,)       theta      Ry        :  0.000000
7     (11,)      (4,)       theta      Ry        :  0.000000
8     (12,)      (5,)       theta      Ry        :  0.000000
9     (14,)      (5,)       theta      Ry        :  0.000000
10    (15,)      (6,)       theta      Ry        :  0.000000
11    (17,)      (6,)       theta      Ry        :  0.000000
12    (18,)      (7,)       theta      Ry        :  0.000000
13    (20,)      (7,)       theta      Ry        :  0.200000
14    (21,)      (8,)       theta      Ry        :  0.300000
15    (23,)      (8,)   

Here's a quick example of how things look a little different if `Gate` objects have multiple parameters:

In [31]:
circuit3 = quasar.Circuit().Ry(0).SO4(0,1)
print(circuit3)
print(circuit3.parameter_str)
for key, parameter_indices in circuit3.parameter_indices.items():
    times, qubits = key
    print('%10r %10r: %r' % (times, qubits, parameter_indices))

T  : |0 |1   |

q0 : -Ry-SO4A-
         |    
q1 : ----SO4B-
              
T  : |0 |1   |

Index Time       Qubits     Name       Gate      :     Value
0     (0,)       (0,)       theta      Ry        :  0.000000
1     (1,)       (0, 1)     A          SO4       :  0.000000
2     (1,)       (0, 1)     B          SO4       :  0.000000
3     (1,)       (0, 1)     C          SO4       :  0.000000
4     (1,)       (0, 1)     D          SO4       :  0.000000
5     (1,)       (0, 1)     E          SO4       :  0.000000
6     (1,)       (0, 1)     F          SO4       :  0.000000

      (0,)       (0,): (0,)
      (1,)     (0, 1): (1, 2, 3, 4, 5, 6)


Now back to our case-study CIS circuit. We'll set the parameters to something deterministic but sensible before proceeding to the rest of the exercise:

In [32]:
parameter_values = []
for I in range(N):
    value = (1.0 - I / 17.0)
    parameter_values.append(+value)
    parameter_values.append(-value)
circuit.set_parameter_values(parameter_values)
print(circuit.parameter_str)

Index Time       Qubits     Name       Gate      :     Value
0     (0,)       (1,)       theta      Ry        :  1.000000
1     (2,)       (1,)       theta      Ry        : -1.000000
2     (3,)       (2,)       theta      Ry        :  0.941176
3     (5,)       (2,)       theta      Ry        : -0.941176
4     (6,)       (3,)       theta      Ry        :  0.882353
5     (8,)       (3,)       theta      Ry        : -0.882353
6     (9,)       (4,)       theta      Ry        :  0.823529
7     (11,)      (4,)       theta      Ry        : -0.823529
8     (12,)      (5,)       theta      Ry        :  0.764706
9     (14,)      (5,)       theta      Ry        : -0.764706
10    (15,)      (6,)       theta      Ry        :  0.705882
11    (17,)      (6,)       theta      Ry        : -0.705882
12    (18,)      (7,)       theta      Ry        :  0.647059
13    (20,)      (7,)       theta      Ry        : -0.647059
14    (21,)      (8,)       theta      Ry        :  0.588235
15    (23,)      (8,)   

## Pauli Expectation Values and Gradients

Let us define $|\Psi (\{ \theta_g \}) \rangle$ as the statevector generated by `circuit` at the current parameter values:

In [33]:
print(circuit)
print(circuit.parameter_str)

T   : |0 |1|2 |3 |4|5 |6 |7|8 |9 |10|11|12|13|14|15|16|17|18|19|20|21|22|23|24|

q0  : -X--@----X---------------------------------------------------------------
          |    |                                                               
q1  : -Ry-Z-Ry-@--@----X-------------------------------------------------------
                  |    |                                                       
q2  : ---------Ry-Z-Ry-@--@----X-----------------------------------------------
                          |    |                                               
q3  : -----------------Ry-Z-Ry-@--@-----X--------------------------------------
                                  |     |                                      
q4  : -------------------------Ry-Z--Ry-@--@-----X-----------------------------
                                           |     |                             
q5  : ----------------------------------Ry-Z--Ry-@--@-----X--------------------
                                       

We can also define a Pauli-sparse Hermitian operator $\hat O$:

In [34]:
I, X, Y, Z = quasar.Pauli.IXYZ()
pauli = quasar.Pauli.zero()
for k in range(N+1):
    pauli += (k + 1) / 10.0 * Z[k]
print(pauli)

+0.1*Z0
+0.2*Z1
+0.3*Z2
+0.4*Z3
+0.5*Z4
+0.6*Z5
+0.7*Z6
+0.8*Z7
+0.9*Z8
+1.0*Z9
+1.1*Z10
+1.2*Z11
+1.3*Z12
+1.4*Z13
+1.5*Z14
+1.6*Z15
+1.7*Z16
+1.8*Z17
+1.9*Z18
+2.0*Z19


A common quantum primitive is to compute the total observable expectation value $O (\{ \theta_g \}) \equiv \langle \Psi (\{ \theta_g \} ) | \hat O | \Psi (\{ \theta_g \})\rangle$ at the current parameter set $\{ \theta_g \}$:

In [37]:
backend1 = quasar.QuasarSimulatorBackend()
start = time.time()
print(backend1.run_pauli_expectation_value(
    circuit=circuit,
    pauli=pauli,
    ))
print('%11.3E s' % (time.time() - start))

(19.49023900463351+0j)
  8.422E+00 s


Another useful quantum primitive is to compute the gradient of the total observable expectation value with respect to the circuit parameters $\frac{\partial O}{\partial \theta_g}$. This can be done in either the ideal infinite sampling limit or with finite sampling by using the parameter shift method [e.g., $\frac{\partial O}{\partial \theta_g} = O (\theta_g + \pi/4) - O(\theta_g - \pi / 4)$ for $\hat R$ gates].

In [18]:
backend1 = quasar.QuasarSimulatorBackend()
start = time.time()
print(backend1.run_pauli_expectation_value_gradient(
    circuit=circuit,
    pauli=pauli,
    ))
print('%11.3E s' % (time.time() - start))

[ 1.02966061+0.j -1.02966061+0.j  0.61804267+0.j -0.61804267+0.j
  0.31802671+0.j -0.31802671+0.j  0.10169844+0.j -0.10169844+0.j
 -0.04323412+0.j  0.04323412+0.j -0.12153042+0.j  0.12153042+0.j
 -0.1358927 +0.j  0.1358927 +0.j -0.09163565+0.j  0.09163565+0.j]
  9.202E-02 s


This can be restricted to user-desired parameter indices to lower the cost, using the `parameter_indices` optional argument:

In [19]:
backend1 = quasar.QuasarSimulatorBackend()
start = time.time()
print(backend1.run_pauli_expectation_value_gradient(
    circuit=circuit,
    pauli=pauli,
    parameter_indices=[0,1,2,3]
    ))
print('%11.3E s' % (time.time() - start))

[ 1.02966061+0.j -1.02966061+0.j  0.61804267+0.j -0.61804267+0.j]
  2.340E-02 s


If ideal infinite sampling is used, the `QuasarUltrafastBackend` can provide substantial savings in time for the gradient computation via efficient analytical gradient methodology. Changing the size of the problem by varying `N` near the top of this workbook shows that substantial gains are possible: 

In [38]:
backend2 = quasar.QuasarUltrafastBackend()
start = time.time()
print(backend2.run_pauli_expectation_value_gradient(
    circuit=circuit,
    pauli=pauli,
    ))
print('%11.3E s' % (time.time() - start))

[ 1.19884403e+00+0.j -1.19884403e+00+0.j  7.37093743e-01+0.j
 -7.37093743e-01+0.j  3.90621909e-01+0.j -3.90621909e-01+0.j
  1.29945373e-01+0.j -1.29945373e-01+0.j -5.85415996e-02+0.j
  5.85415996e-02+0.j -1.80820718e-01+0.j  1.80820718e-01+0.j
 -2.40865570e-01+0.j  2.40865570e-01+0.j -2.45463993e-01+0.j
  2.45463993e-01+0.j -2.07738066e-01+0.j  2.07738066e-01+0.j
 -1.47389265e-01+0.j  1.47389265e-01+0.j -8.61432598e-02+0.j
  8.61432598e-02+0.j -4.00578880e-02+0.j  4.00578880e-02+0.j
 -1.40744357e-02+0.j  1.40744357e-02+0.j -3.46900117e-03+0.j
  3.46900117e-03+0.j -5.36313823e-04+0.j  5.36313823e-04+0.j
 -4.30337293e-05+0.j  4.30337293e-05+0.j -1.18621589e-06+0.j
  1.18621589e-06+0.j  0.00000000e+00+0.j  0.00000000e+00+0.j
 -6.74460645e-18+0.j -1.86482774e-17+0.j]
  1.208E+01 s
